In [72]:
import tensorflow as tf
import tensorflow_hub as hub

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [57]:
model_url = 'https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2'
model = hub.load(model_url)
print('Model Loaded!')

Model Loaded!


In [58]:
model(['I hate ice-cream','I love burgers'])

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[ 0.04303795, -0.01770742, -0.03985476, ..., -0.0442543 ,
         0.0129804 , -0.07323932],
       [ 0.03791416, -0.03689563,  0.04587151, ..., -0.06772752,
         0.06020656, -0.0697788 ]], dtype=float32)>

In [83]:
df = pd.read_csv('movies.csv')
df

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,02/08/2023,129000000,352056482,116.0,Released,Back for seconds.,7.079,1365,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-6...
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,05/04/2023,18000000,65675816,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...
2,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,06/06/2023,200000000,407045464,127.0,Released,Unite or fall.,7.340,1007,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,/woJbg7ZqidhpvqFGGMRhWQNoxwa.jpg,496450-569094-298618-385687-877100-598331-4628...
3,693134,Dune: Part Two,Science Fiction-Adventure,en,Follow the mythic journey of Paul Atreides as ...,4742.163,Legendary Pictures,27/02/2024,190000000,683813734,167.0,Released,Long live the fighters.,8.300,2770,Timothée Chalamet-Zendaya-Rebecca Ferguson-Jav...,epic-based on novel or book-fight-sandstorm-sa...,/czembW0Rk1Ke7lCJGahbOhdCuhV.jpg,/xOMo8BRK7PfcJv9JCnx7s5hj0PX.jpg,438631-763215-792307-1011985-467244-634492-359...
4,640146,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dy...,4425.387,Marvel Studios-Kevin Feige Productions,15/02/2023,200000000,475766228,125.0,Released,Witness the beginning of a new dynasty.,6.507,2811,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kat...,hero-ant-sequel-superhero-based on comic-famil...,/qnqGbB22YJ7dSs4o6M7exTpNxPz.jpg,/m8JTwHFwX7I7JY5fPe4SjqejWag.jpg,823999-676841-868759-734048-267805-965839-1033...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,105444,Doraemon: Nobita's Great Adventure in the Sout...,Animation,ja,The search for documentation about the sea for...,21.805,NaN,07/03/1998,0,0,98.0,Released,NaN,6.739,23,Kazuya Tatekabe-Noriko Ohara-Nobuyo Oyama-Tohr...,anime,/7wAQPPC4XLMvAvwCvR0k6V711my.jpg,/iiZP2Jfj0FAD2ErNaEAEsFBcz7j.jpg,NaN
4996,302401,Snowden,Drama-History-Crime-Thriller,en,CIA employee Edward Snowden leaks thousands of...,21.799,KrautPack Entertainment-Vendian Entertainment-...,15/09/2016,40000000,37357216,134.0,Released,The only safe place is on the run.,7.099,3939,Joseph Gordon-Levitt-Shailene Woodley-Melissa ...,national security agency (nsa)-biography-ideal...,/yfK7zxNL63VWfluFuoUaJj5PdNw.jpg,/mYZYVrQ2ghU3m79q7byeLzNXo1M.jpg,476724-363676-302946-308266-296524-207932-3334...
4997,401773,If It Bleeds We Can Kill It: The Making of 'Pr...,Documentary,en,A behind-the-scenes look at the 1987 sci-fi/ac...,21.794,Automat Pictures,14/03/2001,0,0,29.0,Released,NaN,7.500,25,Arnold Schwarzenegger-Craig R. Baxley-Shane Bl...,predator,/u48Eu6QQDZZimXPlc7ApTJn904P.jpg,/7WMEtFfVA1ycJKIUlvDSUYgfgUH.jpg,NaN
4998,26736,Wizards of Waverly Place: The Movie,Family-TV Movie-Adventure-Comedy-Drama-Fantasy...,en,A young wizard accidentally conjures a spell t...,21.793,NaN,28/08/2009,0,0,94.0,Released,Family vacations can be tricky.,6.600,1282,Selena Gomez-Jake T. Austin-David Henrie-Maria...,sibling 

In [84]:
df = df[['title', 'overview']]
df.head()

,title,overview
0,Meg 2: The Trench,An exploratory dive into the deepest depths of...
1,The Pope's Exorcist,Father Gabriele Amorth Chief Exorcist of the V...
2,Transformers: Rise of the Beasts,When a new threat capable of destroying the en...
3,Dune: Part Two,Follow the mythic journey of Paul Atreides as ...
4,Ant-Man and the Wasp: Quantumania,Super-Hero partners Scott Lang and Hope van Dy...


In [85]:
df.isna().sum()

title        0
overview    54
dtype: int64

In [86]:
df = df.dropna()

In [87]:
df.shape

(4946, 2)

In [88]:
overviews = list(df['overview'])
overviews[:3]

['An exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival.',
 "Father Gabriele Amorth Chief Exorcist of the Vatican investigates a young boy's terrifying possession and ends up uncovering a centuries-old conspiracy the Vatican has desperately tried to keep hidden.",
 'When a new threat capable of destroying the entire planet emerges Optimus Prime and the Autobots must team up with a powerful faction known as the Maximals. With the fate of humanity hanging in the balance humans Noah and Elena will do whatever it takes to help the Transformers as they engage in the ultimate battle to save Earth.']

In [89]:
embeddings = model(overviews)
embeddings.shape

TensorShape([4946, 512])

In [90]:
pca = PCA(n_components=3) # Because we want to see the Embedding Space in 3d!
emb3d = pca.fit_transform(embeddings)

In [91]:
df2 = pd.DataFrame(data=emb3d, columns=['PC1', 'PC2', 'PC3'])
fig = go.Figure(data=[go.Scatter3d(x=df2['PC1'], y=df2['PC2'], z=df2['PC3'], mode='markers')])

fig.update_layout(title='Embedding Space',
                  scene=dict(
                      xaxis=dict(title='PC1'),
                      yaxis=dict(title='PC2'),
                      zaxis=dict(title='PC3')
                  ),
                  width=1000,
                  height=700
                 )
fig.show()

In [92]:
nn = NearestNeighbors(n_neighbors=10)
nn.fit(embeddings)

NearestNeighbors(n_neighbors=10)

In [93]:
def recommender (text):
  emb = model([text])
  neighbors = nn.kneighbors(emb, return_distance=False)[0]
  return df['title'].iloc[neighbors].tolist()

In [97]:
recommender('romance')

['Onsen porno chitai',
 'Tentacles',
 '(500) Days of Summer',
 'The Holiday',
 'Romeo + Juliet',
 'The Kissing Booth',
 'Spread',
 'Lovefully Yours Veda',
 'I Want You',
 'Meet Cute']